In [2]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

# load in data from assets folder
mlb_df=pd.read_csv("assets/mlb.csv")
nhl_df=pd.read_csv("assets/nhl.csv")
nba_df=pd.read_csv("assets/nba.csv")
nfl_df=pd.read_csv("assets/nfl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]


In [5]:
def sports_team_performance(mlb_df=mlb_df, nhl_df=nhl_df, nba_df=nba_df, nfl_df=nfl_df, cities=cities):
    
    # only uses data from 2018
    nfl_df = nfl_df[nfl_df['year'] == 2018]
    mlb_df = mlb_df[mlb_df['year'] == 2018]
    nba_df = nba_df[nba_df['year'] == 2018]
    nhl_df = nhl_df[nhl_df['year'] == 2018]
    
    #contains each sport's dataframe
    cities = cities.set_index('Metropolitan area')
    sports = {'NFL':nfl_df, 'MLB':mlb_df, 'NBA':nba_df, 'NHL':nhl_df}
    
    #used to clean cities df by sport given
    def clean_names(teams, city, sport):
            teams = re.sub('\[(.*?)\]', '', str(teams)).strip()
            if 'NHL' == sport:
                teams = re.findall('(Red Wings)|(Maple Leafs)|(Blue Jackets)|(Golden Knights)|([A-Z]{1}[a-z]{0,20})', teams)
            elif 'NFL' == sport:
                teams = re.findall('(Dummy1)|(Dummy2)|(Dummy3)|([0-9]{2}|[A-Z]{1})([a-z]{0,20})', teams)
            elif 'NBA' == sport:
                teams = re.findall('(Dummy1)|(Dummy2)|(Trail Blazers)|([0-9]{2}|[A-Z]{1})([a-z]{0,20})', teams)
            elif 'MLB' == sport:
                teams = re.findall('(Dummy1)|(Red Sox)|(Blue Jays)|(White Sox)|([A-Z]{1}[a-z]{0,20})', teams)
            out = pd.DataFrame(teams, index=[city for i in range(0,len(teams))])
            return out
    
    #used to get team name by sport df given
    def get_team(item):
            item = re.sub('[*+]', '', item)
            item = re.sub('\((.*?)\)', '', item)
            x = item.split(' ')
            if sport == 'NFL':
                return x[len(x)-1]
            elif sport == 'NHL':
                if x[len(x)-1] == 'Wings':
                    return 'Red Wings'
                elif x[len(x)-1] == 'Leafs':
                    return 'Maple Leafs'
                elif x[len(x)-1] == 'Jackets':
                    return 'Blue Jackets'
                elif x[len(x)-1] == 'Knights':
                    return 'Golden Knights'
                else:
                    return x[len(x)-1]
            elif sport == 'NBA':
                if x[len(x)-1] == 'Blazers':
                    return 'Trail Blazers'
                else:
                    return x[len(x)-1].strip()
            elif sport == 'MLB':
                if x[len(x)-1] == 'Sox':
                    if x[len(x)-2] == 'White':
                        return 'White Sox'
                    else:
                        return 'Red Sox'
                elif x[len(x)-1] == 'Jays':
                    return 'Blue Jays'
                else:
                    return x[len(x)-1]
    
    clean_dfs = {}
    for sport in list(sports.keys()):
        #cleanning cities for sport (dataframe contains city/team name, population, just team name)
        cleaned_cities = pd.concat(list(map(clean_names, cities[sport].copy(), cities[sport].copy().index, [sport for i in range(0,len(cities[sport].copy())-1)]))).rename(columns={0: 'Team'})
        cleaned_cities['Team'] = cleaned_cities['Team']+cleaned_cities[1]+cleaned_cities[2]+cleaned_cities[3]+cleaned_cities[4]
        cleaned_cities = cleaned_cities.drop([1,2,3,4], axis=1)
        cleaned_cities['Population'] = cleaned_cities.index
        cleaned_cities['Population'] = cleaned_cities['Population'].apply(lambda x: cities.loc[x]['Population (2016 est.)[8]'])
        cleaned_cities['city_cities'] = cleaned_cities.index
        cleaned_cities = cleaned_cities.reset_index()
        del cleaned_cities['index']
        
        sports[sport]['Team'] = pd.Series(map(get_team, sports[sport]['team'].copy())) #clean sport df
        clean_dfs[sport] = pd.merge(sports[sport], cleaned_cities, how='inner', left_on='Team', right_on='Team') #merge cleaned cities and sport

    return clean_dfs 

sports_team_performance()

C:\Users\sunda\Anaconda3\lib\site-packages\ipykernel_launcher.py:73: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


{'NFL':     DSRS   L League    MoV  OSRS   PA    PD   PF    SRS   SoS  T   W  W-L%  \
 0    2.1   5    NFL    6.9   3.1  325   111  436    5.2  -1.8  0  11  .688   
 1   -5.2   9    NFL   -7.1  -3.6  433  -114  319   -8.8  -1.7  0   7  .438   
 2   -0.6  10    NFL   -6.6  -6.3  374  -105  269   -6.9  -0.3  0   6  .375   
 3   -5.9  12    NFL   -6.8  -2.0  441  -108  333   -7.8  -1.1  0   4  .250   
 4    6.4   6    NFL    6.4   0.6  287   102  389    7.0   0.6  0  10  .625   
 5    1.7   6    NFL    4.3   3.9  360    68  428    5.6   1.3  1   9  .594   
 6    0.6   8    NFL   -2.1  -1.0  392   -33  359   -0.3   1.7  1   7  .469   
 7   -3.4  10    NFL   -5.4   0.0  455   -87  368   -3.4   2.0  0   6  .375   
 8    1.4   5    NFL    5.4   2.4  316    86  402    3.8  -1.5  0  11  .688   
 9   -0.6   6    NFL    5.6   3.9  344    89  433    3.4  -2.2  0  10  .625   
 10   3.5   7    NFL    0.4  -3.2  303     7  310    0.2  -0.2  0   9  .563   
 11   4.0  11    NFL   -4.4  -8.1  316   -71 

In [14]:
# Calculates Win Loss Ratio correlation with population for MLB 

combined = sports_team_performance()['MLB']
combined['W-L%'] = combined['W-L%'].apply(lambda x: float(x))
combined['Population'] = combined['Population'].astype(float)

fin1 = combined.groupby('city_cities')['W-L%'].agg({'win_loss': np.average}).sort_values(by='city_cities', ascending=True)
fin2 = combined.groupby('city_cities')['Population'].agg({'pop': np.average}).sort_values(by='city_cities', ascending=True)

population_by_region = fin2['pop'] # pass in metropolitan area population from cities
win_loss_by_region = fin1['win_loss'] # pass in win/loss ratio from nfl_df in the same order as cities["Metropolitan area"]

stats.pearsonr(population_by_region, win_loss_by_region)[0] # correlation is first value

C:\Users\sunda\Anaconda3\lib\site-packages\ipykernel_launcher.py:73: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\sunda\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  import sys
C:\Users\sunda\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  


0.15003737475409495